In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta, date
import time
# from reader import feed

In [2]:
tic = time.perf_counter()

today = date.today()
print("Today's date:", today)
days_back = 365*2
rows_drop = days_back-2
start_date = (date.today() - timedelta(days=days_back)).strftime('%Y-%m-%d')
start_date

Today's date: 2021-08-11


'2019-08-12'

In [3]:
tickers = pd.read_csv("Input/tickers_input.csv")
tickers
# tickers = "SPY AAPL MSFT"
# tickers = ['AAPL','MSFT','AMD']

,ETRN,SRRA,MGY,AXSM,GNOG,AMZN,GOOGL,GOOG,BTCM,HEXO,...,WRI,SGOC,BITF,MOMO,RMED,TSLA,PIXY,DRNA,CARV,OLB


In [4]:
# df = yf.download(tickers,start= start_date, end=date.today())
# df.head()

dfout = pd.DataFrame()

for ticker in tickers:
    df = yf.Ticker(ticker)
    df = df.history('2y')
    df.drop(columns=['Dividends','Stock Splits'],inplace=True)
    df['conv_line'] = ((df.High.rolling(9).max())+(df.Low.rolling(9).min()))/2 #checked
    df['base_line'] = ((df.High.rolling(26).max())+(df.Low.rolling(26).min()))/2 #checked
    df['lead_a'] = ((df['conv_line']+df['base_line'])/2).shift(25) # checked
    df['lead_b'] = (((df.High.rolling(52).max())+(df.Low.rolling(52).min()))/2).shift(25) # checked
    df['lag_span'] = df.Close.shift(-25) #checked
    df['crit1'] = (df.conv_line>df.base_line).astype(int)  # conversion line above baseline, checked
    df['crit2'] = (df.lead_a.shift(-25)>df.lead_a.shift(-24)).astype(int)  # lead 1 moving up, checked
    # df['test2'] = df.lead_a.shift(-25)-df.lead_a.shift(-24)  #  this code used to check crit2
    df['crit3'] = (df.Close>df.lead_a).astype(int)  # price above Lead 1, checked
    # df['test3'] = df.Close - df.lead_a # used to check crit3
    df['crit4'] = (df.Close>df.lead_b).astype(int)  # price above Lead 2, checked
    # df['test4'] = df.Close - df.lead_a # used to check crit3
    df['crit5'] = (df.lead_a.shift(-25)>df.lead_b.shift(-25)).astype(int)  # lead 1 above lead 2, checked
    # df['test5'] = df.lead_a.shift(-25)-df.lead_b.shift(-25)  # test for crit4
    df['crit6'] = (((df.lag_span>df.High).shift(25)).fillna(0).astype(int)) # lag line above price action, checked
    # df['test6'] = (df.lag_span - df.High).shift(25) # test for crit6
    df['crit7'] = (df.lead_b.shift(-25)>df.lead_b.shift(-24)).astype(int)  # lead 2 moving up, checked
    # df['test7'] = df.lead_b.shift(-25)-df.lead_b.shift(-24)  #  this code used to check crit7
    df = df.drop(df.columns[[0,1,2,3,4,5,6,7,8,9]],axis=1, inplace=False)  #remove unnecessary columns
    df['total_pts'] = df.sum(axis=1)  # adds up the points
    df = df.drop(df.columns[[0,1,2,3,4,5,6]],axis=1,inplace=False) #remove unnecessary columns
    row_count = len(df.index) #get the row count of the dataframe
    df = df.iloc[row_count-2: , :] #remove all rows except the last two
    chg = df.total_pts.diff() #find difference between the last two rows
    df['chg'] = chg #add the chg data to the dataframe
    row_count = len(df.index) #get the row count of the dataframe
    df = df.iloc[row_count-1: , :] # remove all but the last row
    df['ticker'] = ticker # add ticker label column
    df = df[['ticker','total_pts','chg']] # move ticker column to the front
    # df = pd.DataFrame(aa)
#     dfout = dfout.join[df['total_pts']]
    dfout = dfout.append(df)

dfout.to_csv('Output/check_file.csv')
# print(ticker)
pd.set_option('display.max_rows',999)
dfout

,ticker,total_pts,chg
Date,,,
2021-08-11,ETRN,0,0.0
2021-08-11,SRRA,2,0.0
2021-08-11,MGY,1,0.0
2021-08-11,AXSM,0,0.0
2021-08-11,GNOG,4,0.0
2021-08-11,AMZN,0,0.0
2021-08-11,GOOGL,4,0.0
2021-08-11,GOOG,4,0.0
2021-08-11,BTCM,3,1.0


In [5]:
toc = time.perf_counter()
print(f"Total processing time is {toc-tic:0.4f} seconds")
# count = len(tickers)

# # for i in count:
# df['50MA'] = tickers[0].Close.rolling(50).mean()
# df
# df.to_csv('check_file.csv')

Total processing time is 25.1835 seconds


In [6]:
# for ticker in tickers:
#     df['50MA'] = [ticker].Close.rolling(50).mean()
#     df['200MA'] = [ticker].Close.rolling(200).mean()

# # aa['50MA'] = aa.Close.rolling(50).mean()
# # aa['200MA'] = aa.Close.rolling(200).mean()
# aa['conv_line'] = ((aa.High.rolling(9).max())+(aa.Low.rolling(9).min()))/2 #checked
# aa['base_line'] = ((aa.High.rolling(26).max())+(aa.Low.rolling(26).min()))/2 #checked
# aa['lead_a'] = ((aa['conv_line']+aa['base_line'])/2).shift(25) # checked
# aa['lead_b'] = (((aa.High.rolling(52).max())+(aa.Low.rolling(52).min()))/2).shift(25) # checked
# aa['lag_span'] = aa.Close.shift(-25) #checked
# aa['crit1'] = (aa.conv_line>aa.base_line).astype(int)  # conversion line above baseline, checked
# aa['crit2'] = (aa.lead_a.shift(-25)>aa.lead_a.shift(-24)).astype(int)  # lead 1 moving up, checked
# # aa['test2'] = aa.lead_a.shift(-25)-aa.lead_a.shift(-24)  #  this code used to check crit2
# aa['crit3'] = (aa.Close>aa.lead_a).astype(int)  # price above Lead 1, checked
# # aa['test3'] = aa.Close - aa.lead_a # used to check crit3
# aa['crit4'] = (aa.Close>aa.lead_b).astype(int)  # price above Lead 2, checked
# # aa['test4'] = aa.Close - aa.lead_a # used to check crit3
# aa['crit5'] = (aa.lead_a.shift(-25)>aa.lead_b.shift(-25)).astype(int)  # lead 1 above lead 2, checked
# # aa['test5'] = aa.lead_a.shift(-25)-aa.lead_b.shift(-25)  # test for crit4
# aa['crit6'] = (((aa.lag_span>aa.High).shift(25)).fillna(0).astype(int)) # lag line above price action, checked
# # aa['test6'] = (aa.lag_span - aa.High).shift(25) # test for crit6
# aa['crit7'] = (aa.lead_b.shift(-25)>aa.lead_b.shift(-24)).astype(int)  # lead 2 moving up, checked
# # aa['test7'] = aa.lead_b.shift(-25)-aa.lead_b.shift(-24)  #  this code used to check crit7
# aa = aa.drop(aa.columns[[0,1,2,3,4,5,6,7,8,9]],axis=1, inplace=False)  #remove unnecessary columns
# aa['total_pts'] = aa.sum(axis=1)  # adds up the points
# # df = pd.DataFrame(aa)

# # for index, row in df.iterrows():
# #     print(row['c1'], row['c2'])

# data.to_csv('check_file.csv')
# aa.to_csv('check_file.csv')
# pd.set_option('display.max_rows',999)
# aa